In [1]:
# 常用的引用和创建对象实体
from pyspark import SparkConf,SparkContext
conf = SparkConf().setMaster("local").setAppName("My App")
sc = SparkContext(conf = conf)

# 1, 简介
1. 累加器（accumulator）: 对信息进行聚合
2. 广播变量（broadcast variable）: 高效分发较大的对象
3. 批操作
4. 外部程序的交互方式


- **共享变量是一种可以在Spark任务中使用的特殊类型的变量**
- **当任务需要很长的时间的时候，在你多个数据元素之间共享一次配置就会比较有效率**
- Spark可以使用**pipe()**方法来与其他程序通过标准输入和标准输出进行交互


# 2, 类 